<a href="https://colab.research.google.com/github/Trickshotblaster/nn-practices/blob/main/TrigramLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [138]:
import random
import math

In [139]:
import requests

url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
url = 'https://raw.githubusercontent.com/karpathy/makemore/master/names.txt'
#url = 'https://raw.githubusercontent.com/Trickshotblaster/nn-practices/main/TrigramLM.ipynb'
response = requests.get(url)

with open('file.txt', 'wb') as f:
  f.write(response.content)

In [140]:
with open('file.txt', 'r') as data:
  dataset = data.readlines()

with open('file.txt', 'r') as data:
  raw_data = data.read()


In [141]:
dataset = raw_data.split("\n")

In [142]:
dataset[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [143]:
for i, data in enumerate(dataset):
  dataset[i] = "." + data + "."

In [144]:
dataset[:10]

['.emma.',
 '.olivia.',
 '.ava.',
 '.isabella.',
 '.sophia.',
 '.charlotte.',
 '.mia.',
 '.amelia.',
 '.harper.',
 '.evelyn.']

In [145]:
raw_data = ''.join(dataset)

In [146]:
raw_data[:10]

'.emma..oli'

In [147]:
train_set = dataset[:math.floor(len(dataset) * 0.8)]
dev_set = dataset[math.floor(len(dataset) * 0.8):math.floor(len(dataset) * 0.9)]
test_set = dataset[math.floor(len(dataset) * 0.9):len(dataset) - 1]

In [148]:
vocab = set(raw_data)

In [149]:
vocab

{'.',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [150]:
vocab_len = len(vocab)

In [151]:
vocab_len

27

In [152]:
stoi = {char:i for i, char in enumerate(vocab)}

In [153]:
itos = {i:char for i, char in enumerate(vocab)}

In [154]:
import torch

In [155]:
import torch.nn.functional as functional

In [156]:
# make dataset
xs, ys = [], []
for data in train_set:
  for ch1, ch2, ch3 in zip(data, data[1:], data[2:]):
    xs.append([stoi[ch1], stoi[ch2]])
    ys.append(stoi[ch3])

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
num

314304

In [157]:
ws = torch.randn((vocab_len*2, vocab_len))
ws.requires_grad = True
ws.shape

torch.Size([54, 27])

In [158]:
xenc = functional.one_hot(xs[0], num_classes=vocab_len).float().flatten()
xenc @ ws

tensor([ 2.4417,  0.6242,  0.5274, -0.7158,  0.3234,  0.9105, -1.4854, -1.9854,
         1.0855,  0.2052, -1.0453, -1.9271,  0.4494, -2.0548, -2.5127, -2.6856,
        -0.1722,  1.0604, -1.1879, -1.3265,  0.3905,  0.4823,  0.4962,  0.2256,
        -0.5923, -0.2155,  0.4573], grad_fn=<SqueezeBackward4>)

In [159]:
(xenc @ ws).shape

torch.Size([27])

In [160]:
num_epochs = 100
lr = 20

In [161]:
torch.randn((3, 1))

tensor([[1.1641],
        [0.2010],
        [1.7861]])

In [162]:
for epoch in range(num_epochs):

    xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
    logits = xenc @ ws
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)
    loss = -probs[torch.arange(int(num / 2)), ys].log().mean() + (ws.mean() * 0.01)
    print(loss.item())

    ws.grad = None
    loss.backward()
    ws.data += ws.grad * -lr

4.398468971252441
3.9637348651885986
3.6439754962921143
3.4164071083068848
3.251295804977417
3.1246070861816406
3.023594379425049
2.9414072036743164
2.8735547065734863
2.816770553588867
2.768657922744751
2.7274484634399414
2.691816806793213
2.6607470512390137
2.63344669342041
2.6092886924743652
2.58777117729187
2.568490505218506
2.551116704940796
2.535378932952881
2.52105450630188
2.507956027984619
2.4959280490875244
2.4848387241363525
2.474575996398926
2.4650466442108154
2.456169605255127
2.4478759765625
2.440106153488159
2.432809591293335
2.4259424209594727
2.419466018676758
2.4133458137512207
2.407552719116211
2.4020602703094482
2.3968446254730225
2.3918850421905518
2.387162208557129
2.382659435272217
2.378361463546753
2.374253749847412
2.3703243732452393
2.366560697555542
2.3629531860351562
2.3594913482666016
2.356166124343872
2.3529701232910156
2.349895715713501
2.3469347953796387
2.3440823554992676
2.3413314819335938
2.338677406311035
2.3361141681671143
2.333636999130249
2.331242

In [163]:
xs[:10]

tensor([[12, 21],
        [21, 20],
        [20, 20],
        [20,  9],
        [12, 18],
        [18,  3],
        [ 3, 25],
        [25,  1],
        [ 1, 25],
        [25,  9]])

In [164]:
def forward(xs):
  xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
  logits = xenc @ ws
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdims=True)
  outchar = itos[torch.multinomial(probs, num_samples=1).item()]
  return outchar

def prompt(ins):
  out = ins
  for char in range(500):
    last_two = ins[-2:]
    xs = torch.tensor([[stoi[last_two[0]], stoi[last_two[1]]]])
    outs = forward(xs)
    out += outs
  return out

prompt("Rom")

'Rom.eistiiio.eagbpeai.eieillraiaaaeueyiaiesdaiae.aoiiaiiiaauaaaadaaa..tycyiii.aeeayoaifsa.aiyfpeaa.rai..aaf.aayeuaeiimlu.hyyeiiias.eibiaeaa.aeaa...aaaieeiaaaa.waaaaa.uaiyaaary.aay.i.aa..iaaiaahadpai..aiaafaariiuaaieeylyiwi.ap.ia.aa..i.ieuaaazeiiuea.aaieaaaiieiaef.yayea.a.iypay.asii..baiaiiinsa.i..rieaiiuyai.ayaaa.i.eibioyyaiaaiafiiaaaa.a.aey.iai...idaiafaaaugsiaayamue.ad.aui.ewi.aaaaenae.aieeiaaiaeafey.aaaibia.ia.aoiabeia.bneeeiiia.ai.iaiaieies.iefseiiaieaaauayeee.ea..y.faiah..axeia.iai.eaaaiaaeeaa'

In [165]:
# make test set
xs, ys = [], []
for data in test_set:
  for ch1, ch2, ch3 in zip(data, data[1:], data[2:]):
    xs.append([stoi[ch1], stoi[ch2]])
    ys.append(stoi[ch3])

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
num

39052

In [166]:
xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
logits = xenc @ ws
counts = logits.exp()
probs = counts / counts.sum(1, keepdim=True)
loss = -probs[torch.arange(int(num / 2)), ys].log().mean() + (ws.mean() * 0.01)
print(loss.item())

2.4957985877990723


In [167]:
# make dev set
xs, ys = [], []
for data in dev_set:
  for ch1, ch2, ch3 in zip(data, data[1:], data[2:]):
    xs.append([stoi[ch1], stoi[ch2]])
    ys.append(stoi[ch3])

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
num

38860

In [168]:
xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
logits = xenc @ ws
counts = logits.exp()
probs = counts / counts.sum(1, keepdim=True)
loss = -probs[torch.arange(int(num / 2)), ys].log().mean() + (ws.mean() * 0.01)
print(loss.item())

2.498171806335449


Looks like it might be overfitting? The dev set results seem fine, but the test set loss it much higher than it should be.

In [169]:
# make dev set
dxs, dys = [], []
for data in dev_set:
  for ch1, ch2, ch3 in zip(data, data[1:], data[2:]):
    dxs.append([stoi[ch1], stoi[ch2]])
    dys.append(stoi[ch3])

dxs = torch.tensor(xs)
dys = torch.tensor(ys)
dnum = xs.nelement()
dnum

<ipython-input-169-af72f93a7454>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dxs = torch.tensor(xs)
<ipython-input-169-af72f93a7454>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dys = torch.tensor(ys)


38860

In [170]:
ws = torch.randn((vocab_len*2, vocab_len))
ws.requires_grad = True
ws.shape

torch.Size([54, 27])

In [171]:
num_epochs = 100
lr = 20
regularization = 0.01

In [172]:
last_loss = 2.1

for epoch in range(num_epochs):
    dxenc = functional.one_hot(dxs, num_classes=vocab_len).float().flatten(start_dim=1)
    dlogits = dxenc @ ws
    dcounts = dlogits.exp()
    dprobs = dcounts / dcounts.sum(1, keepdim=True)
    dloss = -dprobs[torch.arange(int(num / 2)), dys].log().mean() + (ws.mean() * regularization)
    dloss.detach()

    xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
    logits = xenc @ ws
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)
    loss = -probs[torch.arange(int(num / 2)), ys].log().mean() + (ws.mean() * regularization * (abs(last_loss - dloss)))

    print(loss.item())
    ws.grad = None
    loss.backward()
    ws.data += ws.grad * -lr

    last_loss = loss.item()

4.185744285583496
3.8254897594451904
3.585930585861206
3.418062925338745
3.292776107788086
3.192509174346924
3.108778476715088
3.0371713638305664
2.975004196166992
2.9204859733581543
2.872347354888916
2.8296351432800293
2.791598320007324
2.7576215267181396
2.7271902561187744
2.699868679046631
2.6752824783325195
2.6531100273132324
2.633068561553955
2.6149086952209473
2.598407745361328
2.583366870880127
2.5696113109588623
2.5569872856140137
2.5453617572784424
2.534620761871338
2.5246663093566895
2.515415668487549
2.506795883178711
2.4987456798553467
2.491211175918579
2.484145164489746
2.477506160736084
2.471256971359253
2.465364694595337
2.4597995281219482
2.454535961151123
2.4495487213134766
2.4448165893554688
2.4403204917907715
2.436042308807373
2.4319663047790527
2.4280776977539062
2.424363613128662
2.4208126068115234
2.417412757873535
2.4141552448272705
2.4110300540924072
2.408029794692993
2.405146598815918
2.402374029159546
2.399704933166504
2.397134304046631
2.3946564197540283
2.39

In [175]:
prompt(".e")

'.elrelzrlnlrlarraraurrlmzaulnrrrrrlvlhrnrbslnzrardylrltrlnalncelzllrovromermuyrveumairlblrrsllnrrrrlqlvllmlezrreurlalllrndmlyrvlllbrdlyratxrmrxsylrlrrzlnlilrimorcrrvmrsmydrllrlzxlmlrriialxrldlrclrzoisaalln.isnlrlrdrrasrltrraoeyelleelrynrrrrlleldarniliedlnllylelrryrlyrtmldlllaslyleiryrlnldsrbrrrl.dlnxyiyylrlhllmramlheedlhnrrlyyislalrrlgdrlalvalueorrulilllrralrrrnlxlrlllnrrraallrosilrrnrlreriarzxelyrvtdrlbllramalrnlrlirmlllrznsesyttrnzrlhlellmcvlarrskrrrreullsdlrlhrmdrvllmlsulrsalazvlrvmatraoalrhayr'

Idk, maybe? I've definitely screwed something up here but I have no idea what